In [122]:
day = 0  # if day = 0, it would be today
month = 8
mode = 'sim' # mode = 'sim' or 'lab'

In [123]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, timedelta
import pickle

In [124]:
dir_path = r"C:/Users/" + os.environ.get("USERNAME") + r"/Desktop/DiGriFlex_Code"
# dir_path = r"/Users/mohammadrayati/Desktop/DiGriFlex_Code"

In [125]:
def figuring(x_arr, y, yax_name, date_arr, y_lag, y_for, y_real):
    if len(y_lag) == 0:
        date_arr1 = []
    else:
        date_arr1 = date_arr
    if len(y_for) == 0:
        date_arr2 = []
    else:
        date_arr2 = date_arr + pd.DateOffset(minutes=10)
    if len(y_real) == 0:
        date_arr3 = []
    else:
        date_arr3 = date_arr
    fig = go.Figure([
    go.Scatter(
        name='Upper Bound',
        x=x_arr,
        y=np.array(y[0, :]) + np.array(y[1, :]),
        mode='lines',
        marker=dict(color='rgba(30, 70, 100, 0.3)'),
        line=dict(width=0),
        showlegend=False
    ),
    go.Scatter(
        name='Lower Bound',
        x=x_arr,
        y=np.array(y[0, :]) - np.array(y[2, :]),
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(30, 70, 100, 0.3)',
        fill='tonexty',
        showlegend=False
    ),
    go.Scatter(
        x=x_arr, y=np.array(y[0, :]),
        line_color='rgb(30, 70, 100)',
        name='mean',
    )])
    fig.update_layout(
        yaxis_title=yax_name,
        hovermode="x",
        width=700,
        height=400
    )
    fig.update_traces(mode='lines')
    fig.add_trace(go.Scatter(x=date_arr1, y=y_lag, mode = "markers", name='realized'))
    fig.add_trace(go.Scatter(x=date_arr2, y=y_for, mode = "markers", name='forecast'))
    fig.add_trace(go.Scatter(x=date_arr3, y=y_real, mode = "markers", name='implemented'))
    fig.show()

In [126]:
date_sp = datetime.now()
if day > 0:
    date_sp = date_sp.replace(day = day)
    date_sp = date_sp.replace(month = month)
date_str = str(date_sp.year) + str(date_sp.month) + str(date_sp.day)
try:
    file_to_read = open(dir_path + r'/Result/for' + date_str + '.pickle', 'rb')
    grid_inp = pickle.load(file_to_read)
    V_mag = pickle.load(file_to_read)
    result_p_pv = pickle.load(file_to_read)
    result_p_dm = pickle.load(file_to_read)
    result_q_dm = pickle.load(file_to_read)
    result_SOC = pickle.load(file_to_read)
    result_price = pickle.load(file_to_read)
    robust_par = pickle.load(file_to_read)
    file_to_read.close()
except:
    result_p_pv = np.empty((3,144))
    result_p_dm = np.empty((3,144))
    result_q_dm = np.empty((3,144))
    result_SOC = np.empty((3,1))
    result_p_pv[:] = np.NaN
    result_p_dm[:] = np.NaN
    result_q_dm[:] = np.NaN
    result_SOC[:] = np.NaN
try:
    file_to_read = open(dir_path + r'/Result/res' + date_str + '.pickle', 'rb')
    P_SC = pickle.load(file_to_read)
    Q_SC = pickle.load(file_to_read)
    RPP_SC = pickle.load(file_to_read)
    RPN_SC = pickle.load(file_to_read)
    RQP_SC = pickle.load(file_to_read)
    RQN_SC = pickle.load(file_to_read)
    SOC_dersired = (np.array(pickle.load(file_to_read)) * 100 / 64).tolist()
    prices_vec = pickle.load(file_to_read)
    file_to_read.close()
except:
    P_SC = np.empty((1,144))
    Q_SC = np.empty((1,144))
    RPP_SC = np.empty((1,144))
    RPN_SC = np.empty((1,144))
    RQP_SC = np.empty((1,144))
    RQN_SC = np.empty((1,144))
    SOC_dersired = np.empty((1,144))
    P_SC[:] = np.NaN
    Q_SC[:] = np.NaN
    RPP_SC[:] = np.NaN
    RPN_SC[:] = np.NaN
    RQP_SC[:] = np.NaN
    RQN_SC[:] = np.NaN
    SOC_dersired[:] = np.NaN


colnames=['0', 'running_time', 'date', 'pvmax_lag', 'irr_lag', 'pdem_lag', 'qdem_lag',
          'pvmax_for', 'irr_for', 'pdem_for', 'qdem_for', 'pv_sim', 'fp_sim', 'fq_sim',
          'pv_real', 'pv_q_real', 'SOC_real', 'fp_real', 'fq_real', 'asked_P', 'asked_Q']
data_rt = pd.read_csv(dir_path + r'/Result/realtime_data.csv', names=colnames, header=None)
data_rt['date'] = pd.to_datetime(data_rt['date'])
data_rt = data_rt[data_rt['date'] >= str(date_sp.year) + '-' + str(date_sp.month) + '-' + str(date_sp.day)]
data_rt = data_rt[data_rt['date'] < str(date_sp.year) + '-' + str(date_sp.month) + '-' + str(date_sp.day + 1)]

x = pd.date_range(pd.Timestamp(date_sp).floor(freq ='D'), periods=144, freq='10min')

date = data_rt['date']
pvmax_lag = data_rt['pvmax_lag'].values
irr_lag = data_rt['irr_lag'].values
pdem_lag = data_rt['pdem_lag'].values
qdem_lag = data_rt['qdem_lag'].values
pvmax_for = data_rt['pvmax_for'].values
irr_for = data_rt['irr_for'].values
pdem_for = data_rt['pdem_for'].values
qdem_for = data_rt['qdem_for'].values
pv_sim = data_rt['pv_sim'].values
fp_sim = - data_rt['fp_sim'].values
fq_sim = - data_rt['fq_sim'].values
pv_real = data_rt['pv_real'].values
pv_q_real = data_rt['pv_q_real'].values
SOC_real = data_rt['SOC_real'].values
fp_real = data_rt['fp_real'].values
fq_real = data_rt['fq_real'].values
asked_P = data_rt['asked_P'].values
asked_Q = data_rt['asked_Q'].values

if mode == 'sim':
    figuring(x, result_p_dm, 'Demand active power (kW)', date, pdem_lag, pdem_for, [])
    figuring(x, result_q_dm, 'Demand reactive power (kVar)', date, qdem_lag, qdem_for, [])
    figuring(x, result_p_pv, 'PV max Production (kW)', date, pvmax_lag, pvmax_for, pv_sim)
    figuring(x, np.array([P_SC, RPP_SC, RPN_SC]), 'Net active power (kW)', date, [], asked_P, fp_sim)
    figuring(x, np.array([Q_SC, RQP_SC, RQN_SC]), 'Net reactive power (kVar)', date, [], asked_Q, fq_sim)
elif mode == 'lab':
    figuring(x, result_p_dm, 'Active power (kW)', date, pdem_lag, pdem_for, [])
    figuring(x, result_q_dm, 'Reactive power (kVar)', date, qdem_lag, qdem_for, [])
    figuring(x, result_p_pv, 'PV max Production (kW)', date, pvmax_lag, pvmax_for, pv_real) 
    figuring(x, np.array([P_SC, RPP_SC, RPN_SC]), 'Net active power (kW)', date, [], asked_P, fp_real)
    figuring(x, np.array([Q_SC, RQP_SC, RQN_SC]), 'Net reactive power (kVar)', date, [], asked_Q, fq_real)
figuring(x, np.array([SOC_dersired, [result_SOC[1]*result_SOC[0]] * 144, [result_SOC[2]*result_SOC[0]] * 144]), 'State of Charge (%)', date, [], [], SOC_real)

In [127]:
%%HTML 
<script>
    function luc21893_refresh_cell(cell) {
        if( cell.luc21893 ) return;
        cell.luc21893 = true;
        console.debug('New code cell found...' );
        
        var div = document.createElement('DIV');            
        cell.parentNode.insertBefore( div, cell.nextSibling );
        div.style.textAlign = 'right';
        var a = document.createElement('A');
        div.appendChild(a);
        a.href='#'
        a.luc21893 = cell;
        a.setAttribute( 'onclick', "luc21893_toggle(this); return false;" );

        cell.style.visibility='hidden';
        cell.style.position='absolute';
        a.innerHTML = '[show code]';
    }
    function luc21893_refresh() {                
        if( document.querySelector('.code_cell .input') == null ) {            
            // it apeears that I am in a exported html
            // hide this code
            var codeCells = document.querySelectorAll('.jp-InputArea')
            codeCells[0].style.visibility = 'hidden';
            codeCells[0].style.position = 'absolute';                        
            for( var i = 1; i < codeCells.length; i++ ) {
                luc21893_refresh_cell(codeCells[i].parentNode)
            }
            window.onload = luc21893_refresh;
        }                 
        else {
            // it apperas that I am in a jupyter editor
            var codeCells = document.querySelectorAll('.code_cell .input')
            for( var i = 0; i < codeCells.length; i++ ) {
                luc21893_refresh_cell(codeCells[i])
            }            
            window.setTimeout( luc21893_refresh, 1000 )
        }        
    }
    
    function luc21893_toggle(a) {
        if( a.luc21893.style.visibility=='hidden' ) {
            a.luc21893.style.visibility='visible';        
            a.luc21893.style.position='';
            a.innerHTML = '[hide code]';
        }
        else {
            a.luc21893.style.visibility='hidden';        
            a.luc21893.style.position='absolute';
            a.innerHTML = '[show code]';
        }
    }
    
    luc21893_refresh()
</script>